In [117]:
import sagemaker
import boto3
from datetime import datetime
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')



from pyathena import connect

In [118]:
sess = sagemaker.Session() #Intialzie session

bucket = sess.default_bucket()
RD_Bucket = 'policedatasetbucket' #Raw Data
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
s3 = boto3.client('s3') # Create an S3 client
s3_resource = boto3.resource('s3') # Create an S3 resource


print('Bucket - > ',bucket)
print('Role - > ',role)
print('Region - > ',region)
print('S3 - > ',s3)
print('S3 Resource - > ',s3_resource)

Bucket - >  sagemaker-us-east-1-859074047513
Role - >  arn:aws:iam::859074047513:role/LabRole
Region - >  us-east-1
S3 - >  <botocore.client.S3 object at 0x7f7f35dd33d0>
S3 Resource - >  s3.ServiceResource()


In [119]:
# Function to verify if bucket exist, if not create
def verify_create_bucket(bucket_name):
    response = s3.list_buckets()
    for bucket in response['Buckets']:
        if bucket['Name'] == bucket_name:
            print(f"The {bucket_name} bucket exists.")
            break
    else:
        print(f"The {bucket_name} bucket does not exist, creating")
        !aws s3 mb s3://{bucket}/

In [120]:
verify_create_bucket(bucket)

The sagemaker-us-east-1-859074047513 bucket exists.


In [121]:
#function to download yearly data
# def Get_Data(year):
#     url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
#     df = pd.read_csv(url)
#     return df

In [122]:
# List of years since 2018
# Years = list(range(2018,datetime.now().year+1))
# Years

In [123]:
#iterate over years
# for year in Years:
#     year_df = Get_Data(year)
#     csv_buffer = StringIO()
#     year_df.to_csv(csv_buffer, index=False)
#     file = 'SDPD_Calls_' + str(year) + '.csv'
#     s3_resource.Object(bucket,file).put(Body=csv_buffer.getvalue())
#     print(file,'loaded in',bucket,'bucket')

In [124]:
# def URL_2_Bucket(url,file_name):
#     df = pd.read_csv(url, low_memory=False)
#     csv_buffer = StringIO()
#     df.to_csv(csv_buffer, index=False)
#     file = file_name + '.csv'
#     s3_resource.Object(RD_Bucket,file).put(Body=csv_buffer.getvalue())
#     print(file,'loaded in',RD_Bucket,'bucket')
    

In [125]:
# type_url = f"http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv"
# dipo_url = f"http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv"
# ripa_stops_url = "https://seshat.datasd.org/pd/ripa_stops_datasd.csv"
# ripa_stops_dic = "https://seshat.datasd.org/pd/ripa_stops_dictionary_datasd.csv"

In [126]:
# URL_2_Bucket(type_url,'Type')
# URL_2_Bucket(dipo_url,'Dispo')
# URL_2_Bucket(ripa_stops_url,'Ripa_Stops')
# URL_2_Bucket(ripa_stops_dic,'Ripa_Stops_Dic')

In [127]:
ingest_create_athena_db_passed = False

# Create Athena Database

In [128]:
database_name = "sd_police_db"

In [129]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [130]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)


In [131]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS sd_police_db


In [132]:
import pandas as pd

pd.read_sql(statement, conn)

""


In [133]:
# Show databases
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,sd_police_db


# Drop Database if Needed

In [134]:
# drop_db_name = "dsoaw"
# drop_db_query = f"DROP DATABASE IF EXISTS {drop_db_name}"

# df_show = pd.read_sql(drop_db_query, conn)
# df_show.head()

# Drop Table if Needed

In [135]:
drop_table_name = "table_sd_ripa" 

drop_table_query = f"DROP TABLE IF EXISTS {database_name}.{drop_table_name}"

df_show1 = pd.read_sql(drop_table_query, conn)
df_show1.head()

""


In [136]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [137]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


# S3 Bucket Folder Information for 2019

In [138]:
# s3_private_path_csv19 = "s3://{}/SanDiego-Police-Dataset/policedataset19/csv".format(bucket)
# print(s3_private_path_csv19)

In [139]:
# table_name4 = "table_sd_2019"

In [140]:
# # SQL statement to execute
# statement_4 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
# incident_num string,
# date_time string,
# day_of_week string,
# address_number_primary string,
# address_dir_primary string,
# address_road_primary string,
# address_sfx_primary string,
# address_dir_intersecting string,
# address_road_intersecting string,
# address_sfx_intersecting string,
# call_type string,
# disposition string,
# beat string,
# priority string) 

# ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
# TBLPROPERTIES ('skip.header.line.count'='1')""".format(database_name, table_name4, s3_private_path_csv19)

# print(statement_4)

In [141]:
# import pandas as pd

# pd.read_sql(statement_4, conn)

In [142]:
# database_name = "sd_police_db"
# statement = "SHOW TABLES in {}".format(database_name)

# df_show = pd.read_sql(statement, conn)

# df_show.head(6)

In [143]:
# pd.read_sql(statement_4, conn)

# df_2019 = pd.read_sql(f"SELECT * FROM {database_name}.{table_name4}", conn)

In [144]:
# df_2019

# S3 Bucket Folder Information for 2021

In [145]:
s3_private_path_csv = "s3://{}/SanDiego-Police-Dataset/policedatasetsd/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-859074047513/SanDiego-Police-Dataset/policedatasetsd/csv


In [146]:
table_name = "table_sd_2021"

In [147]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(database_name, table_name, s3_private_path_csv)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS sd_police_db.table_sd_2021(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION 's3://sagemaker-us-east-1-859074047513/SanDiego-Police-Dataset/policedatasetsd/csv'
TBLPROPERTIES ('skip.header.line.count'='1')


In [148]:
import pandas as pd

pd.read_sql(statement, conn)

""


# Varify Tables in a Specified Database

In [149]:
database_name = "sd_police_db"
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)

df_show.head(5)

,tab_name
0,table_sd_2021
1,table_sd_2022
2,table_sd_2023


In [150]:
pd.read_sql(statement, conn)

df_2021 = pd.read_sql(f"SELECT * FROM {database_name}.{table_name}", conn)

# S3 Bucket Folder Information for 2022

In [151]:
s3_private_path_csv1 = "s3://{}/SanDiego-Police-Dataset/policedatasetsd22/csv".format(bucket)
print(s3_private_path_csv1)

s3://sagemaker-us-east-1-859074047513/SanDiego-Police-Dataset/policedatasetsd22/csv


In [152]:
table_name1 = "table_sd_2022"

In [153]:
# SQL statement to execute
statement_1 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(database_name, table_name1, s3_private_path_csv1)

print(statement_1)

CREATE EXTERNAL TABLE IF NOT EXISTS sd_police_db.table_sd_2022(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION 's3://sagemaker-us-east-1-859074047513/SanDiego-Police-Dataset/policedatasetsd22/csv'
TBLPROPERTIES ('skip.header.line.count'='1')


In [154]:
import pandas as pd

pd.read_sql(statement_1, conn)

""


In [155]:
database_name = "sd_police_db"
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,table_sd_2021
1,table_sd_2022
2,table_sd_2023


In [156]:
pd.read_sql(statement_1, conn)

df_2022 = pd.read_sql(f"SELECT * FROM {database_name}.{table_name1}", conn)

In [157]:
df_2022

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority
0,E22060041419,2022-06-29 11:42:21,4,0,None,01ST,AVE,None,CEDAR,None,415W,DUP,526,0
1,E22060041420,2022-06-29 11:43:50,4,3800,None,JEWELL,ST,None,None,None,5150,A,122,2
2,E22060041422,2022-06-29 11:44:26,4,3600,None,DALBERGIA,ST,None,None,None,CC,R,511,3
3,E22060041423,2022-06-29 11:44:52,4,2500,None,EL CAJON,BLV,None,None,None,415,K,813,2
4,E22060041425,2022-06-29 11:45:32,4,2400,None,TOSCA,WAY,None,None,None,1171,K,621,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499251,E22060041414,2022-06-29 11:38:55,4,4100,None,54TH,PL,None,None,None,594,R,822,2
499252,E22060041415,2022-06-29 11:38:57,4,5000,None,SANTA MONICA,AVE,None,None,None,242,R,614,1
499253,E22060041416,2022-06-29 11:40:20,4,4800,None,DEL MAR,AVE,None,None,None,INFO,K,614,2
499254,E22060041417,2022-06-29 11:41:47,4,300,None,05TH,AVE,None,None,None,1186,K,523,3


# S3 Bucket Folder Information for 2023

In [158]:
s3_private_path_csv2 = "s3://{}/SanDiego-Police-Dataset/policedatasetsd23/csv".format(bucket)
print(s3_private_path_csv2)

s3://sagemaker-us-east-1-859074047513/SanDiego-Police-Dataset/policedatasetsd23/csv


In [159]:
table_name2 = "table_sd_2023"

In [160]:
# SQL statement to execute
statement_2 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1',
                'serialization.null.format'='')""".format(database_name, table_name2, s3_private_path_csv2)

print(statement_2)

CREATE EXTERNAL TABLE IF NOT EXISTS sd_police_db.table_sd_2023(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION 's3://sagemaker-us-east-1-859074047513/SanDiego-Police-Dataset/policedatasetsd23/csv'
TBLPROPERTIES ('skip.header.line.count'='1',
                'serialization.null.format'='')


In [161]:
import pandas as pd

pd.read_sql(statement_2, conn)

""


In [162]:
database_name = "sd_police_db"
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,table_sd_2021
1,table_sd_2022
2,table_sd_2023


In [163]:
pd.read_sql(statement_2, conn)

df_2023 = pd.read_sql(f"SELECT * FROM {database_name}.{table_name2}", conn)

In [164]:
df_2023

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority
0,E23010004824,2023-01-04 15:59:54,4,5500,None,OTAY MESA,RD,None,None,None,586,O,725,4
1,E23010004825,2023-01-04 16:00:10,4,500,None,13TH,ST,None,None,None,415,K,521,2
2,E23010004826,2023-01-04 16:00:37,4,0,None,48TH,ST,None,TROJAN,None,CW,K,826,1
3,E23010004827,2023-01-04 16:00:50,4,6800,None,CARTHAGE,ST,None,None,None,415V,A,322,1
4,E23010004828,2023-01-04 16:01:24,4,2700,None,WORDEN,ST,None,None,None,415,K,613,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99806,E23030022386,2023-03-15 23:25:50,4,2500,None,IMPERIAL,AVE,None,None,None,INFO,K,512,2
99807,E23030022400,2023-03-15 23:44:05,4,800,None,27TH (SB),ST,None,None,None,5150,K,724,1
99808,E23030022401,2023-03-15 23:44:23,4,14700,None,VALLE DEL SUR,CT,None,None,None,459A,CAN,937,2
99809,E23030022404,2023-03-15 23:46:26,4,400,None,17TH,ST,None,None,None,SELENF,K,521,3


# S3 Bucket Folder Information for RIPA Dataset

In [165]:
# s3_private_path_csv_ripa = "s3://{}/SanDiego-Police-Dataset/ripapolicedatasetsd/csv".format(bucket)
# print(s3_private_path_csv_ripa)

In [166]:
# table_name3 = "table_sd_ripa"

In [167]:
# # SQL statement to execute
# statement_3 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
# `stop_id` string,
# `ori` string,
# `agency` string,
# `exp_years` string,
# `date_stop` string,
# `time_stop` string,
# `stopduration` string,
# `stop_in_response_to_cfs` string,
# `office_assignment_key` string,
# `assignment` string,
# `intersection` string,
# `address_block` string,
# `land_mark` string,
# `address_street` string,
# `highway_exit` string,
# `isschool` string,
# `school_name` string,
# `address_city` string,
# `beat` string,
# `beat_name` string,
# `pid` string,
# `isstudent` string,
# `perceived_limited_english` string,
# `perceived_age` string,
# `perceived_gender` string,
# `gender_nonconforming` string,
# `gend` string,
# `gend_nc` string,
# `perceived_lgbt` string
# ) 

# ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
# WITH SERDEPROPERTIES ("separatorChar" = ",")
# LOCATION '{}'
# TBLPROPERTIES ('skip.header.line.count'='1',
#                'serialization.null.format'='')""".format(database_name, table_name3, s3_private_path_csv_ripa)


# print(statement_3)

In [168]:
# import pandas as pd

# pd.read_sql(statement_3, conn)

In [169]:
# database_name = "sd_police_db"
# statement = "SHOW TABLES in {}".format(database_name)

# df_show = pd.read_sql(statement, conn)
# df_show.head(5)

In [170]:
# pd.read_sql(statement_3, conn)

# ripa_df = pd.read_sql(f"SELECT * FROM {database_name}.{table_name3}", conn)

# Join The Three datasets 2021, 2022, 2023

In [171]:
police_call_df = pd.read_sql(f"SELECT * FROM {database_name}.{table_name} UNION ALL \
                               SELECT * FROM {database_name}.{table_name1} UNION ALL \
                               SELECT * FROM {database_name}.{table_name2} LIMIT 20", conn)

In [172]:
police_call_df

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority
0,E21060046805,2021-06-28 07:36:28,2,5200,None,52ND,PL,None,None,None,SELENF,O,826,3
1,E21060046806,2021-06-28 07:36:33,2,3400,None,SPORTS ARENA,BLV,None,None,None,459A,K,611,3
2,E21060046807,2021-06-28 07:37:42,2,6200,None,MADELINE,ST,None,None,None,586,O,821,2
3,E21060046808,2021-06-28 07:38:22,2,1700,None,02ND,AVE,None,None,None,FU,O,529,2
4,E21060046809,2021-06-28 07:40:14,2,9200,None,AERO,DR,None,None,None,COURT,K,311,4
5,E21060046810,2021-06-28 07:41:35,2,4000,None,MENLO,AVE,None,None,None,SELENF,O,831,3
6,E21060046811,2021-06-28 07:42:12,2,9300,None,GALVIN,AVE,None,None,None,SELENF,K,242,3
7,E21060046813,2021-06-28 07:42:59,2,2500,None,MORENA,BLV,None,None,None,459R,R,116,3
8,E21060046814,2021-06-28 07:43:08,2,1400,None,02ND,AVE,None,None,None,FU,O,526,2
9,E21060046815,2021-06-28 07:43:41,2,1300,None,J,ST,None,None,None,C6,O,521,2


In [173]:
# main_df = pd.read_sql(f"SELECT * FROM (SELECT * FROM {database_name}.{table_name} UNION ALL \
#                                        SELECT * FROM {database_name}.{table_name1} UNION ALL \
#                                        SELECT * FROM {database_name}.{table_name2}) Q1 LEFT JOIN (SELECT beat, beat_name, COUNT(beat) Incident_count FROM {database_name}.{table_name3} \
#                                        WHERE date_stop >= 2021-01-01 GROUP BY beat, beat_name) Q2 ON Q1.beat = Q2.beat", conn) 

### Checking the data types, removing missing values, duplicated values

In [174]:
# shape of the dataframes
print(f'shape of the dataframe police dataframe {df_2021.shape}')
print(f'shape of the dataframe police dataframe {df_2022.shape}')
print(f'shape of the dataframe police dataframe {df_2023.shape}')

shape of the dataframe police dataframe (568947, 14)
shape of the dataframe police dataframe (499256, 14)
shape of the dataframe police dataframe (99811, 14)


In [175]:
# concatenate the tables 
# join_df = pd.concat([df_2021, df_2022, df_2023], axis=0)
# join_df

In [176]:
join_df["beat"].nunique()

130

In [177]:
join_df.describe(include='all')

,incident_num,date_time,day_of_week,address_number_primary,address_road_primary,address_sfx_primary,call_type,disposition,beat,priority
count,1063538,1063538,1.063538e+06,1.063538e+06,1063538,1063538,1063538,1063538,1.063538e+06,1063538.0
unique,1063538,1055165,NaN,NaN,8321,42,249,19,NaN,7.0
top,E21120032396,2021-05-12 00:30:55,NaN,NaN,IMPERIAL,ST,415,K,NaN,2.0
freq,1,3,NaN,NaN,21265,346804,95928,559023,NaN,470285.0
first,NaN,2021-01-01 00:00:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2023-03-15 23:53:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,4.048911e+00,3.356213e+03,NaN,NaN,NaN,NaN,4.965265e+02,NaN
std,NaN,NaN,1.962039e+00,3.249948e+03,NaN,NaN,NaN,NaN,2.354804e+02,NaN
min,NaN,NaN,1.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,-1.000000e+00,NaN
25%,NaN,NaN,2.000000e+00,6.000000e+02,NaN,NaN,NaN,NaN,3.130000e+02,NaN


In [178]:
# Data types 
join_df.dtypes

incident_num                      object
date_time                 datetime64[ns]
day_of_week                        int64
address_number_primary             int64
address_road_primary              object
address_sfx_primary               object
call_type                         object
disposition                       object
beat                               int64
priority                        category
dtype: object

In [179]:
from datetime import datetime

join_df["date_time"] = pd.to_datetime(join_df["date_time"])
join_df

,incident_num,date_time,day_of_week,address_number_primary,address_road_primary,address_sfx_primary,call_type,disposition,beat,priority
0,E21010000001,2021-01-01 00:00:11,6,5000,CHAPARRAL,WAY,AU1,W,326,1
1,E21010000002,2021-01-01 00:00:14,6,500,05TH,AVE,1186,CAN,523,3
2,E21010000003,2021-01-01 00:00:26,6,3600,38TH,ST,AU1,DUP,839,1
3,E21010000004,2021-01-01 00:00:31,6,2700,WORDEN,ST,AU1,W,613,1
4,E21010000006,2021-01-01 00:01:20,6,800,SAN DIEGO,PL,INFO,W,121,2
...,...,...,...,...,...,...,...,...,...,...
99806,E23030022386,2023-03-15 23:25:50,4,2500,IMPERIAL,AVE,INFO,K,512,2
99807,E23030022400,2023-03-15 23:44:05,4,800,27TH (SB),ST,5150,K,724,1
99808,E23030022401,2023-03-15 23:44:23,4,14700,VALLE DEL SUR,CT,459A,CAN,937,2
99809,E23030022404,2023-03-15 23:46:26,4,400,17TH,ST,SELENF,K,521,3


In [180]:
join_df.dtypes

incident_num                      object
date_time                 datetime64[ns]
day_of_week                        int64
address_number_primary             int64
address_road_primary              object
address_sfx_primary               object
call_type                         object
disposition                       object
beat                               int64
priority                        category
dtype: object

In [181]:
join_df.sort_values(by="date_time", ascending = True)

,incident_num,date_time,day_of_week,address_number_primary,address_road_primary,address_sfx_primary,call_type,disposition,beat,priority
0,E21010000001,2021-01-01 00:00:11,6,5000,CHAPARRAL,WAY,AU1,W,326,1
1,E21010000002,2021-01-01 00:00:14,6,500,05TH,AVE,1186,CAN,523,3
2,E21010000003,2021-01-01 00:00:26,6,3600,38TH,ST,AU1,DUP,839,1
3,E21010000004,2021-01-01 00:00:31,6,2700,WORDEN,ST,AU1,W,613,1
4,E21010000006,2021-01-01 00:01:20,6,800,SAN DIEGO,PL,INFO,W,121,2
...,...,...,...,...,...,...,...,...,...,...
99806,E23030022386,2023-03-15 23:25:50,4,2500,IMPERIAL,AVE,INFO,K,512,2
99807,E23030022400,2023-03-15 23:44:05,4,800,27TH (SB),ST,5150,K,724,1
99808,E23030022401,2023-03-15 23:44:23,4,14700,VALLE DEL SUR,CT,459A,CAN,937,2
99809,E23030022404,2023-03-15 23:46:26,4,400,17TH,ST,SELENF,K,521,3


In [182]:
join_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1063538 entries, 0 to 99810
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   incident_num            1063538 non-null  object        
 1   date_time               1063538 non-null  datetime64[ns]
 2   day_of_week             1063538 non-null  int64         
 3   address_number_primary  1063538 non-null  int64         
 4   address_road_primary    1063538 non-null  object        
 5   address_sfx_primary     1063538 non-null  object        
 6   call_type               1063538 non-null  object        
 7   disposition             1063538 non-null  object        
 8   beat                    1063538 non-null  int64         
 9   priority                1063538 non-null  category      
dtypes: category(1), datetime64[ns](1), int64(3), object(5)
memory usage: 114.4+ MB


Our first file contains 1,168,014 rows and 14 columns.</br>
Out of 14 columns 4 are int type, 8 are strings/object, and 1 is datetime type.</br>
Out of 14 columns 8 columns have null/missing values. 


In [183]:
# check null values
join_df.isna().sum()

incident_num              0
date_time                 0
day_of_week               0
address_number_primary    0
address_road_primary      0
address_sfx_primary       0
call_type                 0
disposition               0
beat                      0
priority                  0
dtype: int64

In [184]:
from tqdm import tqdm

def Data_Quality_Report(df):

    #Initial table
    freqDF = pd.DataFrame(columns=['Feature',
                                   'Mode',
                                   'Mode Freq.',
                                   'Mode %',
                                   '2nd Mode',
                                   '2nd Mode Freq.',
                                   '2nd Mode %'])
    for col in tqdm(df.columns):
        try:
            #print(col)
            freq = df[col].value_counts()
            freqdf = freq.to_frame()
            fRow = freqdf.iloc[0]
            #try:
            secRow = freqdf.iloc[1]
            #except:
            #secRow = 0
            fPrct = fRow[0] / len(df[col])
            #try:
            secPrct = secRow[0] / len(df[col])
            #except:
                #secPrct = 0
            try:
                mode1 = int(fRow.name)
            except:
                mode1 = fRow.name
            try:
                mode2 = int(secRow.name)
            except:
                try:
                    mode2 = secRow.name
                except:
                    mode2 = 0
            freqDF = freqDF.append({'Feature':col,
                                    'Mode':mode1,
                                    'Mode Freq.':fRow[0],
                                    'Mode %':fPrct,\
                                    '2nd Mode':mode2,
                                    '2nd Mode Freq.':secRow[0],
                                    '2nd Mode %':secPrct},
                                    ignore_index=True)
        except:
            pass

    freqDF = freqDF.set_index('Feature')

    #Nulls, Counts, Cardinality
    NUllFeatures = round(df.isnull().sum() / df.shape[0],4)\
          .sort_values(ascending=False)
    Count = df.count()
    uni = df.nunique()

    #Formating
    NUllFeatures.to_frame(name="% Miss.")
    Count.to_frame(name="Count")
    uni.to_frame()
    result = pd.concat([Count, NUllFeatures,uni], axis=1)
    result.columns =["Count","% Miss.","Card."]
    result = pd.concat([result, freqDF], axis=1)
    """
    result = result.style.format({'% Miss.': "{:.1%}",
                         'Mode %': "{:.0%}",
                         '2nd Mode %': "{:.0%}",
                         'Count': "{:,}",
                         'Card.': "{:,}",
                         'Mode Freq.': "{:,}",
                        '2nd Mode Freq.': "{:,}"})"""
    return result

In [185]:
DQR_Calls = Data_Quality_Report(join_df)
DQR_Calls

100%|██████████| 10/10 [00:03<00:00,  3.22it/s]


,Count,% Miss.,Card.,Mode,Mode Freq.,Mode %,2nd Mode,2nd Mode Freq.,2nd Mode %
incident_num,1063538,0.0,1063538,E21120032396,1,9.402579e-07,E21110014041,1,9.402579e-07
date_time,1063538,0.0,1055165,2021-05-12 00:30:55,3,2.820774e-06,2022-04-28 18:40:37,3,2.820774e-06
day_of_week,1063538,0.0,7,5,158928,1.494333e-01,6,158097,1.486520e-01
address_number_primary,1063538,0.0,250,0,179246,1.685375e-01,4000,22628,2.127616e-02
address_road_primary,1063538,0.0,8321,IMPERIAL,21265,1.999458e-02,05TH,18923,1.779250e-02
address_sfx_primary,1063538,0.0,42,ST,346804,3.260852e-01,AVE,275383,2.589310e-01
call_type,1063538,0.0,249,415,95928,9.019706e-02,SELENF,64404,6.055637e-02
disposition,1063538,0.0,19,K,559023,5.256258e-01,O,159217,1.497050e-01
beat,1063538,0.0,130,521,49445,4.649105e-02,122,44165,4.152649e-02
priority,1063538,0.0,7,2,470285,4.421892e-01,3,237198,2.230273e-01


As we can see the `address_sfx_intersecting` and `address_dir_intersecting` have no values and we can drop these columns.</br>
Also the `address_dir_primary` and `address_road_intersecting` have more than 80% missing values; therefore we can consider removing these columns too. 

### Dropping columns with more than 60% missing values

In [186]:
# Removing unnecessary columns
def drop_columns_with_max_missing_values(df):
    mis_var = [x for x in df.columns if df[x].isnull().sum() > 0]
    df[mis_var].isnull().sum()

    limit = np.abs((df.shape[0] * 0.6))
    var_to_be_dropped = [x for x in mis_var if df[x].isnull().sum() > limit]
    print('Columns with more than 60% missing values: \n\n', var_to_be_dropped)

    df.drop(columns=var_to_be_dropped, axis=1, inplace=True)
    return df

df = drop_columns_with_max_missing_values(join_df)

Columns with more than 60% missing values: 

 []


In [187]:
join_df.shape

(1063538, 10)

We dropped the unnecesssary columns from the dataset

In [188]:
# check duplicated values
join_df.duplicated().sum()

0

We don't have any duplicated values in dataset

In [189]:
join_df.isnull().sum()

incident_num              0
date_time                 0
day_of_week               0
address_number_primary    0
address_road_primary      0
address_sfx_primary       0
call_type                 0
disposition               0
beat                      0
priority                  0
dtype: int64

In [190]:
join_df = join_df.dropna(axis=0)


In [191]:
join_df.isnull().sum()

incident_num              0
date_time                 0
day_of_week               0
address_number_primary    0
address_road_primary      0
address_sfx_primary       0
call_type                 0
disposition               0
beat                      0
priority                  0
dtype: int64

In [192]:
join_df.dtypes

incident_num                      object
date_time                 datetime64[ns]
day_of_week                        int64
address_number_primary             int64
address_road_primary              object
address_sfx_primary               object
call_type                         object
disposition                       object
beat                               int64
priority                        category
dtype: object

# Bias Exploration 

***Priority Zero***: Dispatch Immediately. Priority E calls involve an imminent
threat to life. Examples include: officer or person down, no detail accidents and
attempted suicide.

***Priority On***e: Dispatch Immediately. Priority One calls involve serious crimes
in progress or a threat to life. Examples include: missing children, child abuse,
domestic violence, disturbances involving weapons/violence and bomb threats.

***Priority Two***: Dispatch as quickly as possible. Priority Two calls involve
complaints regarding less serious crimes in which there is no threat to life.
Examples include: prowlers who have left, preserve the peace, crime reports for
residents standing by at an inconvenient location, blocked driveway when the
caller is waiting to leave, injured animals, loud parties with mitigating
circumstances.

***Priority Three***: Dispatch as quickly as possible. Priority Three calls involve
minor crimes or requests for service which are not urgent. Examples include:
investigating a cold crime, loud parties involving noise only.

***Priority Four***: Dispatch when no higher priority calls are waiting. Priority
Four calls involve minor requests for police service. Examples include: found
property, most parking violations, etc.

***Priority Nine***: Calls that are formatted for the Telephone Report Unit (TRU).
Priority Nine calls involve requests for police service, which have limited or no
suspect information. Examples include: petty theft, vehicle break-in and false
use of another’s identity.

The dispatch priority system is designed to be only a guide as a higher or lower
priority may be more appropriate.

In [193]:
join_df["priority"] = join_df["priority"].astype("category")

Since we don't have priority `5` in our dictionary and it's not interpreted we can remove from our dataset. 

In [194]:
priority_5 = join_df[join_df["priority"]==5]
priority_5

,incident_num,date_time,day_of_week,address_number_primary,address_road_primary,address_sfx_primary,call_type,disposition,beat,priority
14852,E22070013676,2022-07-10 04:03:59,1,600,K,ST,488R,K,521,5
80897,E22010038843,2022-01-28 03:55:45,6,11400,NORTH TORREY PINES,RD,1186,R,126,5
82492,E22010040535,2022-01-29 05:45:53,7,11400,NORTH TORREY PINES,RD,1186,K,126,5
78716,E23020036542,2023-02-25 09:00:49,7,1000,10TH,AVE,SELENF,K,-1,5


In [ ]:
join_df_clean = join_df.drop(join_df[join_df["priority"]==5].index) 

In [ ]:
join_df_clean

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

join_df_clean.groupby("priority").count()["incident_num"].plot(
    kind="bar", title="Breakdown by Priority Rating")

plt.xlabel("Priority")
plt.ylabel("Incident Count")

In [ ]:
counts_pri = join_df_clean["priority"].value_counts()
counts_pri

In [ ]:
# Change the `priority` values to categorical

# create a dictionary to map old values to new values
priority_map = {0: 'Dispatch immediately', 1: 'Dispatch immediately', 2: 'Dispatch', 3: 'Dispatch', 4: 'non-urgent', 9: 'non-urgent'}
join_df_clean["priority"] = join_df_clean["priority"].replace(priority_map)
join_df_clean

In [ ]:
counts_priority = join_df_clean["priority"].value_counts()
counts_priority

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

join_df_clean.groupby("priority").count()["incident_num"].plot(
    kind="bar", title="Breakdown by Priority Rating")

plt.xlabel("Priority")
plt.ylabel("Incident Count")

# RIPA dataset

In [ ]:
# display(ripa_df)

Somtimes null values are not detectable in the dataframe, so there's a possibility that the empty values are not actually null values. Instead, they may be empty string (`''`), whitespace characters, or other non-null values. So, we decided to replace all empty strings in the dataframe with `NaN` values. 

In [ ]:
# ripa_df.replace('', np.nan, inplace=True)

In [ ]:
# ripa_df.head(5)

In [ ]:
# ripa_df.info()

In [ ]:
# ripa_df.describe()

<!-- ### Check null values -->

In [ ]:
# missing_values = [x for x in ripa_df.columns if ripa_df[x].isnull().sum()>0]
# ripa_df[missing_values].isnull().sum()

In [ ]:
# DQR_RIPA = Data_Quality_Report(ripa_df)
# DQR_RIPA

### Dropping columns with more than 60% missing values

In [ ]:
# Removing unnecessary columns
# drop_columns_with_max_missing_values(ripa_df)

Great! we dropped the columns that included more than 60% missing values. 

In [ ]:
# ripa_df.isnull().sum()

In [ ]:
# ripa_df = ripa_df.dropna(axis=0)

In [ ]:
# ripa_df.isnull().sum()

### change the data types of the columns that should be an integer or float

In [ ]:
# integer_columns = ["stop_id", "exp_years", "stopduration", "stop_in_response_to_cfs", "office_assignment_key", "address_block", "isschool", "beat", "pid", "isstudent", "perceived_limited_english", "perceived_age", \
#                    "gender_nonconforming", "gend"]

In [ ]:
# function to convert the data type of selected columns
# def convert_columns_to_integer(df, columns):
#     df[columns] = df[columns].astype(int)
#     return df

In [ ]:
# convert_columns_to_integer(ripa_df, integer_columns)

In [ ]:
# ripa_df.dtypes

## **Basic Statistical Concepts**

* **Mean**: The mean is one of the measures of central tendency. Simply put, the mean is the average of the values in the given set. The observed values are totaled and divided by the total number of observations to determine the mean.
If $x_i$ is $i^{th}$ observation then mean of all $x_i$ ranging from $ 1\leq i\leq n $ denoted by $\bar x$ is given as

$$ \bar{x} = \sum_{i=1}^{n}\frac{x_i}{n} $$


* **Variance**: Variance is a measure of variation. It is calculated by averaging the squared deviations from the mean.
The degree of spread in your data set is indicated by variation. The greater the spread of the data, the greater the variance in proportion to the mean.
Here's the formula for variance of a sample.

$$S^2 = \frac{\sum_{i=1}^{n}(x_i-\bar x)^2}{n-1}$$


* **Standard Deviation**: The standard deviation is a measure that shows how much variation (such as spread, dispersion, and spread) exists from the mean. The standard deviation represents a "typical" departure from the mean. It is a popular measure of variability since it returns to the data set's original units of measurement.
Here's the formula for standard deviation of a sample.

$$S = \sqrt \frac{\sum_{i=1}^{n}(x_i-\bar x)^2}{n-1}$$